In [5]:
import xml.etree.ElementTree as ElementTree

class XmlListConfig(list):
    def __init__(self, aList):
        for element in aList:
            if element:
                # treat like dict
                if len(element) == 1 or element[0].tag != element[1].tag:
                    self.append(XmlDictConfig(element))
                # treat like list
                elif element[0].tag == element[1].tag:
                    self.append(XmlListConfig(element))
            elif element.text:
                text = element.text.strip()
                if text:
                    self.append(text)


class XmlDictConfig(dict):
    def __init__(self, parent_element):
        if parent_element.items():
            self.updateShim( dict(parent_element.items()) )
        for element in parent_element:
            if len(element):
                aDict = XmlDictConfig(element)
            #   if element.items():
            #   aDict.updateShim(dict(element.items()))
                self.updateShim({element.tag: aDict})
            elif element.items():    # items() is specialy for attribtes
                elementattrib= element.items()
                if element.text:           
                    elementattrib.append((element.tag,element.text ))     # add tag:text if there exist
                self.updateShim({element.tag: dict(elementattrib)})
            else:
                self.updateShim({element.tag: element.text})

    def updateShim (self, aDict ):
        for key in aDict.keys():   # keys() includes tag and attributes
            if key in self:
                value = self.pop(key)
                if type(value) is not list:
                    listOfDicts = []
                    listOfDicts.append(value)
                    listOfDicts.append(aDict[key])
                    self.update({key: listOfDicts})
                else:
                    value.append(aDict[key])
                    self.update({key: value})
            else:
                self.update({key:aDict[key]})



In [7]:
!pip install xmltodict

In [19]:
import xml.etree.ElementTree as ElementTree

import xmltodict
import pprint
import json
import pandas as pd
import numpy as np



#tree = ElementTree.parse('sample.xml')
tree = ElementTree.parse('./raw/BUILT.30.experienced_plans.xml')
root = tree.getroot()
xmldict = XmlDictConfig(root)

xmldict['person']


[{'id': '0',
  'plan': {'score': '-132.96673121955368',
   'selected': 'yes',
   'leg': [{'mode': 'bike',
     'dep_time': '09:03:00',
     'trav_time': '01:18:16',
     'route': {'type': 'generic',
      'start_link': '226193928_0',
      'end_link': '420514148_1',
      'trav_time': '01:18:16',
      'distance': '19569.08484854245'}},
    {'mode': 'bike',
     'dep_time': '13:31:16',
     'trav_time': '00:05:11',
     'route': {'type': 'generic',
      'start_link': '420514148_1',
      'end_link': '458166896_5',
      'trav_time': '00:05:11',
      'distance': '1298.736213671756'}},
    {'mode': 'bike',
     'dep_time': '13:42:27',
     'trav_time': '00:09:46',
     'route': {'type': 'generic',
      'start_link': '458166896_5',
      'end_link': '588168464_0',
      'trav_time': '00:09:46',
      'distance': '2445.0770476881826'}},
    {'mode': 'bike',
     'dep_time': '18:52:13',
     'trav_time': '01:20:43',
     'route': {'type': 'generic',
      'start_link': '588168464_0',
   

In [21]:
#initiate activity related list object 
id_data_act = []
activity_data = []
start_time = []
end_time = []
for type_tag in root.findall('person'):
    for act in type_tag.findall('plan/activity'):
        id_data_act.append(type_tag.get('id')) 
        activity_data.append(act.get('type'))
        start_time.append(act.get('start_time'))
        end_time.append(act.get('end_time'))
activity_data

['Work',
 'Work',
 'Secondary',
 'Work',
 'Home',
 'Home',
 'Work',
 'Home',
 'Secondary',
 'Work',
 'Home',
 'Work',
 'Home',
 'Secondary',
 'Work',
 'Home',
 'Secondary',
 'Work',
 'Home',
 'Work',
 'Home',
 'Secondary',
 'Work',
 'Home',
 'Secondary',
 'Work',
 'Home',
 'Secondary',
 'Secondary',
 'Secondary',
 'Secondary',
 'Home',
 'Home',
 'Work',
 'Home',
 'Secondary',
 'Work',
 'Home',
 'Secondary',
 'Secondary',
 'Secondary',
 'Secondary',
 'Home',
 'Home',
 'Work',
 'Home',
 'Home',
 'Secondary',
 'Work',
 'Secondary',
 'Home',
 'Home',
 'Secondary',
 'Secondary',
 'Secondary',
 'Secondary',
 'Home',
 'Home',
 'Work',
 'Home',
 'Work',
 'Home',
 'Home',
 'Work',
 'Home',
 'Secondary',
 'Work',
 'Home',
 'Work',
 'Home',
 'Home',
 'Secondary',
 'Home',
 'Secondary',
 'Home',
 'Work',
 'Work',
 'Home',
 'Secondary',
 'Home',
 'Work',
 'Home',
 'Home',
 'Work',
 'Home',
 'Secondary',
 'Work',
 'Home',
 'Home',
 'Work',
 'Secondary',
 'Secondary',
 'Home',
 'Home',
 'Work',
 'Hom

In [13]:
#initiate mode related list object 

id_data_mode = []
mode_data = []

for type_tag in root.findall('person'):
    for i ,mode in enumerate(type_tag.findall('plan/leg')):
        id_data_mode.append(type_tag.get('id')) 
        mode_data.append(mode.get('mode'))
           

In [14]:
len(id_data_act)- len(id_data_mode) #exactly shows all ID has one less mode row (due to they arrive final destination)

7536

In [22]:
actList =  list(zip(id_data_act, activity_data, start_time, end_time))
modeList = list(zip(id_data_mode, mode_data))

In [24]:
df_act = pd.DataFrame(actList, columns=['id','activity','start_time','end_time'])

In [25]:
df_act.head()

,id,activity,start_time,end_time
0,0,Work,None,09:03:00
1,0,Work,10:21:16,13:31:16
2,0,Secondary,13:36:27,13:42:27
3,0,Work,13:52:13,18:52:13
4,0,Home,20:12:56,None


In [27]:
df_mode = pd.DataFrame(modeList, columns=['id','mode'])

In [28]:
df_mode.head()

,id,mode
0,0,bike
1,0,bike
2,0,bike
3,0,bike
4,1,car


In [31]:
merge = pd.merge(df_act, df_mode, how='left', left_on='id', right_on='id')
df_unique = merge.drop_duplicates(subset=['id','activity','start_time','end_time'],keep='first')

In [32]:
merge.head() #lots of duplicate due to left join

,id,activity,start_time,end_time,mode
0,0,Work,None,09:03:00,bike
1,0,Work,None,09:03:00,bike
2,0,Work,None,09:03:00,bike
3,0,Work,None,09:03:00,bike
4,0,Work,10:21:16,13:31:16,bike


In [33]:
df_unique.head(10) #remove duplicates and we are done!

,id,activity,start_time,end_time,mode
0,0,Work,None,09:03:00,bike
4,0,Work,10:21:16,13:31:16,bike
8,0,Secondary,13:36:27,13:42:27,bike
12,0,Work,13:52:13,18:52:13,bike
16,0,Home,20:12:56,None,bike
20,1,Home,None,06:03:00,car
21,1,Work,06:24:23,11:53:23,car
22,1007981,Home,None,07:52:00,walk
24,1007981,Secondary,08:14:39,08:19:39,walk
26,1007981,Work,12:37:14,22:54:14,walk


### Calculate the trave time for each trip

In [35]:
# change the type of time
df_unique['start_time']=pd.to_datetime(df_unique['start_time'])
df_unique['end_time']=pd.to_datetime(df_unique['end_time'])

# Calculate the travel time
trave_time = pd.DataFrame(columns={'id','travel_time'})

for i in range(0,len(df_unique)-1):
    if(df_unique.iloc[i]['id']==df_unique.iloc[i+1]['id']):
        diff = pd.Timedelta(df_unique.iloc[i]['end_time']  - df_unique.iloc[i+1]['start_time'] ).seconds / 3600
    else:
        diff = np.nan
    
    trave_time=trave_time.append([{'id': df_unique.iloc[i]['id'], 'travel_time':diff}], ignore_index=True)

#merge 

df_final =  pd.concat([df_unique, trave_time['travel_time']], axis=1)
df_final


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,id,activity,start_time,end_time,mode,travel_time
0,0,Work,NaT,2019-04-29 09:03:00,bike,22.695556
1,NaN,NaN,NaT,NaT,NaN,23.913611
2,NaN,NaN,NaT,NaT,NaN,23.837222
3,NaN,NaN,NaT,NaT,NaN,22.654722
4,0,Work,2019-04-29 10:21:16,2019-04-29 13:31:16,bike,NaN
5,NaN,NaN,NaT,NaT,NaN,23.643611
6,NaN,NaN,NaT,NaT,NaN,NaN
7,NaN,NaN,NaT,NaT,NaN,23.622500
8,0,Secondary,2019-04-29 13:36:27,2019-04-29 13:42:27,bike,19.706944
9,NaN,NaN,NaT,NaT,NaN,NaN


### Calculate the Total travel time of each ID

In [19]:
# Group by
total_travel_time = df_final.groupby(['id']).sum()
total_travel_time



NameError: name 'df_final' is not defined

### Export Excel File

In [20]:
df_final.to_excel("10th_mode_final.xlsx")
total_travel_time.to_excel("10th_totalTime_final.xlsx")
print("export succeed")


NameError: name 'df_final' is not defined

all done
